In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("datafinal.csv", index_col = 0)
data_init = data.transpose()

Construisons la matrice de données X ∈ R<sup>p*c</sup> c étant le nombre de caractéristiques extraites :

### __Calculons de la moyenne__

In [3]:
def calcul_de_moyenne():
    mean = []
    for col in range (0, data.shape[1]):
        colonne = data.iloc[:, col]
        valeur = [1, 2, 3, 4, 5]
        moyenne = (colonne * valeur).sum()/colonne.sum()
        mean.append(moyenne)
    return mean

In [4]:
mean = calcul_de_moyenne()
data_init["moyenne"] = mean

### __Calculons le nombre de personne ayant voté__

In [5]:
data_init["total_vote"] = data.sum(axis = 0)

### __Calculons l'écart-type des scores__

In [6]:
def calcul_ecart_type():
    ecart = []
    for col in range (0, data.shape[1]):
        colonne = data.iloc[:, col]
        valeur = [1, 2, 3, 4, 5]
        ecart_type = ((colonne*(valeur - mean[col])**2 ).sum()/colonne.sum())**(0.5)
        ecart.append(ecart_type)
    return ecart

In [7]:
ecart = calcul_ecart_type()
data_init["Ecart-type"] = ecart

### __Calculons la médiane des scores___

In [8]:
data_init["meadiane"] = data.median(axis = 0)

###  __Calculons le nombre de fois que livre n'a pas été apprécié__

In [9]:
data_init["pas_apprecie"] = data_init.iloc[:, 0] + data_init.iloc[:, 1]

###  __Calculons le nombre de fois que livre a été apprécié__

In [10]:
data_init["apprecie"] = data_init.iloc[:, 3] + data_init.iloc[:, 4]

###  __Calculons le nombre de fois que le choix a été neutre__

In [11]:
data_init["neutre"] = data_init.iloc[:, 2] 

__Remarque__ : Pour avancer nous allons prendre un échantillon de notre base de données pour avancer

In [15]:
data_init["categorie"] = 0

In [22]:
for col in range(0,data_init.shape[0]):
    maximun = max(data_init["pas_apprecie"][col], data_init["apprecie"][col], data_init["neutre"][col])
    if data_init["neutre"][col] == maximun:
        data_init["categorie"][col] = 0
    elif data_init["pas_apprecie"][col] == maximun:
        data_init["categorie"][col] = 1
    else:
        data_init["categorie"][col] = 2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
data_init["categorie"].value_counts()

2    351341
1      9121
0      7520
Name: categorie, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
data, rest = train_test_split(data_init, test_size = 0.99)

In [ ]:
target = data["categorie"]
data = data.drop("catgorie", axis = 1)

Effectuons une segmentation basée sur les k-moyennes avec k = 3
avec :

* Nous commencons par construire __la matrice de similitude basée sur la distance euclidienne__

In [ ]:
def calcul_dist_eucl():
    n = data.shape[0] 
    data_eucl = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            data_eucl[i,j] = np.sqrt(np.sum((data.iloc[i, :] - data.iloc[j, :])**2))
    data_eucl = pd.DataFrame(data_eucl)
    return data_eucl
data_eucl = calcul_dist_eucl()

In [ ]:
data_eucl.to_csv("data_eucl.csv")

 * __Calcul de la matrice de similarité en utilisant la similarité euclidienne__

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
data_cosine = pd.DataFrame(cosine_similarity(data, dense_output = True ))
data_cosine.to_csv("data_cosine.csv")

In [1]:
from sklearn.preprocessing import StandardScaler

In [ ]:
data_eucl_scaler = StandardScaler().fit_transform(data_eucl)
data_cosine_scaler = StandardScaler().fit_transform(data_cosine)

### __Kmeans avec les distances euclidiennes__

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_1 = KMeans(n_clusters = 3)
kmeans_1.fit(data_eucl_scaler)
y_1 = kmeans_1.labels_

### __Kmeans avec la similarité cosinus__

In [ ]:
kmeans_2 = KMeans(n_clusters = 3)
kmeans_2.fit(data_cosine_scaler)
y_2 = kmeans_2.labels_



Utilisons l'analyse en composante principale pour projecter nos segments suivant les deux premières composantes principales avec :
* __La distance euclidienne__

In [ ]:
from sklearn.decomposition import PCA
mypca_1 = PCA(n_components=2)
pca_1 = mypca_1.fit_transform(data_eucl_scaler)

In [ ]:
pca_df_eucl = pd.DataFrame({
    "Dim1" : pca_1[:, 0],
    "Dim2" : pca_1[:, 1],
    "Target" : y_1
})

In [ ]:
sns.scatterplot(data = pca_df_eucl, x = "Dim1", y = "Dim2", hue = 'Target')

In [ ]:
mypca_2 = PCA(n_components=2)
pca_2 = mypca_2.fit_transform(data_cosine_scaler)

In [ ]:
pca_df_cosine = pd.DataFrame({
    "Dim1" : pca_2[:, 0],
    "Dim2" : pca_2[:, 1],
    "Target" : y_2
})

In [ ]:
sns.scatterplot(data = pca_df_cosine, x = "Dim1", y = "Dim2", hue = "Target")

Effectuons une segmentation par clustering spectral sur deux dimensions
* __Distance euclidienne__

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
spectral_cluster_1 = SpectralClustering(n_clusters = 3, affinity ='nearest_neighbors') 
spectral_cluster_1.fit(pca_df_eucl)

In [ ]:
pca_df_eucl["Target"] = spectral_cluster_1.labels_

In [ ]:
sns.scatterplot(data = pca_df_eucl, x = "Dim1", y = "Dim2", hue = "Target")

In [ ]:
spectral_cluster_2 = SpectralClustering(n_clusters = 3,affinity ='nearest_neighbors') 
spectral_cluster_2.fit(pca_df_cosine)

In [ ]:
pca_df_cosine["Target"] = spectral_cluster_2.labels_
sns.scatterplot(data = pca_df_cosine, x = "Dim1", y = "Dim2", hue = "Target")

Calculons la silhouette et l'information mutuelle
 #### __cas spectral__
* Matrice de similarité euclidienne

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_mutual_info_score
target_true = np.zeros((y_spectral_eucl.shape[0]))
y_spectral_eucl = pca_df_eucl["Target"]
X_spectral_eucl = pca_df_eucl.drop("Target", axis = 1)

* **Silhouette**

In [ ]:
silhouette_score(X = X_spectral_eucl, labels = y_spectral_eucl)

* **Information Mutuelle**

In [ ]:
adjusted_mutual_info_score(target_true, y_spectral_eucl)

* __Matrice de similarité cosinus__

In [ ]:
y_spectral_cos = pca_df_cosine["Target"]
X_spectral_cos = pca_df_cosine.drop("Target", axis = 1)

* **Silhouette**

In [ ]:
silhouette_score(X = X_spectral_cos, labels = y_spectral_cos)

* **Information mutuelle**

In [ ]:
adjusted_mutual_info_score(target_true, y_spectral_cos)

### __Partie B__

In [ ]:
data_1 = data_init[data_init["categorie"] == 0][0:200]
data_2 = data_init[data_init["categorie"] == 1][0:200]
data_3 = data_init[data_init["categorie"] == 2][0:200]

In [ ]:
data_4 = pd.concat([data_1, data_2, data_3], axis = 0)
target = data_4["categorie"]
data1 = data_4.drop('categorie', axis =1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
data = pd.DataFrame(cosine_similarity(data, dense_output = True ))

In [ ]:
data_scaler = StandardScaler().fit_transform(data)

In [ ]:
from sklearn.decomposition import PCA
mypca = PCA(n_components=2)
pca = mypca.fit_transform(data_scaler)

In [ ]:
pca_df = pd.DataFrame({
    "Dim1" : pca[:, 0],
    "Dim2" : pca[:, 1],
    "Target" : target
})

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data = pca_df, x = 'Dim1', y = "Dim2", hue = "Target")

In [ ]:
def calcul_dist_eucl():
    n = data1.shape[0] 
    data_eucl = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            data_eucl[i,j] = np.sqrt(np.sum((data1.iloc[i, :] - data1.iloc[j, :])**2))
    data_eucl = pd.DataFrame(data_eucl)
    return data_eucl
data_eucl = calcul_dist_eucl()

In [ ]:
data_eucl_scaler = StandardScaler().fit_transform(data_eucl)

In [ ]:
from sklearn.decomposition import PCA
mypca = PCA(n_components=2)
pca = mypca.fit_transform(data_eucl_scaler)

In [ ]:
pca_eucl_df = pd.DataFrame({
    "Dim1" : pca[:, 0],
    "Dim2" : pca[:, 1],
    "Target" : target
})

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data = pca_df, x = 'Dim1', y = "Dim2", hue = "Target")